In [1]:
HIDDEN_SIZE = 100
N_LAYERS = 2
BATCH_SIZE = 64
N_EPOCHS = 20

In [2]:
from torch.utils.data import Dataset, DataLoader
import csv
import json
import re
import os

class Mydataset(Dataset):
    def __init__(self, train=True):
        if train:
            print('loading trainig dataset')
            file = 'nn_train_poetry.csv'
        else:
            print('loading testing dataset')
            file = 'nn_test_poetry.csv'
        with open(file, 'r', newline='', encoding='utf-8') as myFile:  
            rdr = csv.reader(myFile)
            temp = list(rdr)
            self.x = [a[0][0:200] for a in temp]
            self.y = [a[1] for a in temp]
        self.len = len(self.x)
        self.labels = list(sorted(set(self.y)))
        self.alphabet = list(sorted(set(['+','-','\t'])))
        
    def __getitem__(self, index):
        return self.x[index], self.y[index]
    
    def __len__(self):
        return self.len
        
    def get_labels(self):
        return self.labels
    
    def get_alphabet(self):
        return self.alphabet
    
    def get_alphabet_id(self, c):
        return self.alphabet.index(c)
    
    def get_label(self, id):
        return self.labels[id]
    
    def get_label_id(self, label):
        return self.labels.index(label)

In [3]:
train_dataset = Mydataset()
test_dataset = Mydataset(train=False)
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=BATCH_SIZE, shuffle=True)

test_loader = DataLoader(dataset=test_dataset,
                         batch_size=BATCH_SIZE, shuffle=False)

loading trainig dataset
loading testing dataset


In [4]:
import string

ALL_LETTERS = train_dataset.get_alphabet()
N_CHARS = len(ALL_LETTERS)
N_LABELS = len(train_dataset.get_labels())
print('alphabet size: {}\nclass size: {}'.format(N_CHARS, N_LABELS))

alphabet size: 3
class size: 2


In [5]:
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

def create_variable(tensor):
    return Variable(tensor)

def make_variables(inputs, labels):
    sequence_and_length = [str2ascii_arr(input) for input in inputs]
    vectorized_seqs = [sl[0] for sl in sequence_and_length]
    seq_lengths = torch.LongTensor([sl[1] for sl in sequence_and_length])
    return pad_sequences(vectorized_seqs, seq_lengths, labels)

def str2ascii_arr(msg):
    arr = [ALL_LETTERS.index(c) for c in msg]
    return arr, len(arr)

def pad_sequences(vectorized_seqs, seq_lengths, labels):
    seq_tensor = torch.zeros((len(vectorized_seqs), seq_lengths.max())).long()
    for idx, (seq, seq_len) in enumerate(zip(vectorized_seqs, seq_lengths)):
        seq_tensor[idx, :seq_len] = torch.LongTensor(seq)
    seq_lengths, perm_idx = seq_lengths.sort(0, descending=True)
    seq_tensor = seq_tensor[perm_idx]
    target = labels2tensor(labels)
    if len(labels):
        target = target[perm_idx]
    return create_variable(seq_tensor), \
        create_variable(seq_lengths), \
        create_variable(target)
        
def labels2tensor(labels):
    label_ids = [train_dataset.get_label_id(
        label) for label in labels]
    return torch.LongTensor(label_ids)

In [6]:
class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1, bidirectional=True):
        super(RNNClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        self.n_directions = int(bidirectional) + 1
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          bidirectional=bidirectional)
        self.fc = nn.Linear(hidden_size, output_size)
    def forward(self, input, seq_lengths):
        input = input.t()
        batch_size = input.size(1)
        hidden = self._init_hidden(batch_size)
        embedded = self.embedding(input)
        gru_input = pack_padded_sequence(
            embedded, seq_lengths.data.cpu().numpy())
        self.gru.flatten_parameters()
        output, hidden = self.gru(gru_input, hidden)
        fc_output = self.fc(hidden[-1])
        return fc_output

    def _init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers * self.n_directions,
                             batch_size, self.hidden_size)
        return create_variable(hidden)
    
classifier = RNNClassifier(N_CHARS, HIDDEN_SIZE, N_LABELS, N_LAYERS)

In [7]:
import torch.nn.functional as F
log_interval = 1
test_interval = 100

def train(model, train_loader, test_loader):
    optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)
    model.train()
    steps = 0
    for epoch in range(1, N_EPOCHS+1):
        batch = 0
        for inputs, label in train_loader:
            input, seq_lengths, target = make_variables(inputs, label)
            optimizer.zero_grad()
            output = classifier(input, seq_lengths)
            loss = F.cross_entropy(output, target)
            loss.backward()
            optimizer.step()
            steps += 1
            if steps % log_interval == 0:
                corrects = (torch.max(output, 1)[1].view(target.size()).data == target.data).sum()
                accuracy = 100.0 * corrects/target.shape[0]
                print(
                    '\rBatch[{}] - loss: {:.6f}  acc: {:.4f}%({}/{})'.format(steps,
                                                                             loss.data[0], 
                                                                             accuracy,
                                                                             corrects,
                                                                             target.shape[0]))
            if steps % test_interval == 0:
                eval(test_loader, model)

def eval(test_loader, classifier):
    classifier.eval()
    print("Evaluating trained model ...")
    correct = 0
    train_data_size = len(test_loader.dataset)
    for inputs, labels in test_loader:
        input, seq_lengths, target = make_variables(inputs, labels)
        output = classifier(input, seq_lengths)
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
    print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, train_data_size, 100. * correct / train_data_size))

In [8]:
try:
    train(classifier, train_loader, test_loader)
except KeyboardInterrupt:
    print('\n' + '-' * 89)
    print('Exiting from training early')

F:\settings\anaconda\lib\site-packages\ipykernel\__main__.py:24: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


Batch[1] - loss: 0.692477  acc: 53.0000%(34/64)
Batch[2] - loss: 0.672633  acc: 60.0000%(39/64)
Batch[3] - loss: 0.817655  acc: 40.0000%(26/64)
Batch[4] - loss: 0.696524  acc: 46.0000%(30/64)
Batch[5] - loss: 0.628618  acc: 68.0000%(44/64)
Batch[6] - loss: 0.770143  acc: 56.0000%(36/64)
Batch[7] - loss: 0.975626  acc: 42.0000%(27/64)
Batch[8] - loss: 0.732906  acc: 56.0000%(36/64)
Batch[9] - loss: 0.729844  acc: 53.0000%(34/64)
Batch[10] - loss: 0.676564  acc: 57.0000%(37/64)
Batch[11] - loss: 0.694620  acc: 50.0000%(32/64)
Batch[12] - loss: 0.693218  acc: 57.0000%(37/64)
Batch[13] - loss: 0.694245  acc: 54.0000%(35/64)
Batch[14] - loss: 0.676887  acc: 56.0000%(36/64)
Batch[15] - loss: 0.711457  acc: 48.0000%(31/64)
Batch[16] - loss: 0.730607  acc: 46.0000%(30/64)
Batch[17] - loss: 0.680972  acc: 56.0000%(36/64)
Batch[18] - loss: 0.666372  acc: 59.0000%(38/64)
Batch[19] - loss: 0.697053  acc: 51.0000%(33/64)
Batch[20] - loss: 0.690008  acc: 54.0000%(35/64)
Batch[21] - loss: 0.682348  a

Batch[166] - loss: 0.602369  acc: 70.0000%(45/64)
Batch[167] - loss: 0.499577  acc: 78.0000%(50/64)
Batch[168] - loss: 0.493970  acc: 76.0000%(49/64)
Batch[169] - loss: 0.508791  acc: 79.0000%(51/64)
Batch[170] - loss: 0.562518  acc: 73.0000%(47/64)
Batch[171] - loss: 0.558234  acc: 70.0000%(45/64)
Batch[172] - loss: 0.513959  acc: 82.0000%(53/64)
Batch[173] - loss: 0.484478  acc: 76.0000%(49/64)
Batch[174] - loss: 0.592575  acc: 75.0000%(48/64)
Batch[175] - loss: 0.564011  acc: 71.0000%(46/64)
Batch[176] - loss: 0.527070  acc: 75.0000%(48/64)
Batch[177] - loss: 0.548400  acc: 73.0000%(47/64)
Batch[178] - loss: 0.507420  acc: 78.0000%(50/64)
Batch[179] - loss: 0.624671  acc: 64.0000%(41/64)
Batch[180] - loss: 0.490966  acc: 75.0000%(12/16)
Batch[181] - loss: 0.422290  acc: 81.0000%(52/64)
Batch[182] - loss: 0.530989  acc: 76.0000%(49/64)
Batch[183] - loss: 0.447628  acc: 79.0000%(51/64)
Batch[184] - loss: 0.661369  acc: 59.0000%(38/64)
Batch[185] - loss: 0.514285  acc: 76.0000%(49/64)


Batch[328] - loss: 0.456079  acc: 78.0000%(50/64)
Batch[329] - loss: 0.387507  acc: 79.0000%(51/64)
Batch[330] - loss: 0.355566  acc: 93.0000%(15/16)
Batch[331] - loss: 0.299717  acc: 87.0000%(56/64)
Batch[332] - loss: 0.648250  acc: 65.0000%(42/64)
Batch[333] - loss: 0.369443  acc: 81.0000%(52/64)
Batch[334] - loss: 0.356299  acc: 85.0000%(55/64)
Batch[335] - loss: 0.421478  acc: 85.0000%(55/64)
Batch[336] - loss: 0.262622  acc: 90.0000%(58/64)
Batch[337] - loss: 0.489617  acc: 73.0000%(47/64)
Batch[338] - loss: 0.541062  acc: 76.0000%(49/64)
Batch[339] - loss: 0.521157  acc: 75.0000%(48/64)
Batch[340] - loss: 0.492500  acc: 81.0000%(52/64)
Batch[341] - loss: 0.286555  acc: 89.0000%(57/64)
Batch[342] - loss: 0.412258  acc: 79.0000%(51/64)
Batch[343] - loss: 0.455132  acc: 75.0000%(48/64)
Batch[344] - loss: 0.459419  acc: 78.0000%(50/64)
Batch[345] - loss: 0.563192  acc: 67.0000%(43/64)
Batch[346] - loss: 0.403703  acc: 85.0000%(55/64)
Batch[347] - loss: 0.384242  acc: 82.0000%(53/64)


Batch[491] - loss: 0.333467  acc: 85.0000%(55/64)
Batch[492] - loss: 0.432385  acc: 79.0000%(51/64)
Batch[493] - loss: 0.284313  acc: 87.0000%(56/64)
Batch[494] - loss: 0.318990  acc: 87.0000%(56/64)
Batch[495] - loss: 0.400277  acc: 82.0000%(53/64)
Batch[496] - loss: 0.312658  acc: 87.0000%(56/64)
Batch[497] - loss: 0.378636  acc: 81.0000%(52/64)
Batch[498] - loss: 0.435080  acc: 76.0000%(49/64)
Batch[499] - loss: 0.236386  acc: 90.0000%(58/64)
Batch[500] - loss: 0.461008  acc: 78.0000%(50/64)
Evaluating trained model ...

Test set: Accuracy: 157/208 (75%)

Batch[501] - loss: 0.386515  acc: 82.0000%(53/64)
Batch[502] - loss: 0.359872  acc: 78.0000%(50/64)
Batch[503] - loss: 0.314172  acc: 87.0000%(56/64)
Batch[504] - loss: 0.337637  acc: 84.0000%(54/64)
Batch[505] - loss: 0.456527  acc: 79.0000%(51/64)
Batch[506] - loss: 0.481016  acc: 71.0000%(46/64)
Batch[507] - loss: 0.294910  acc: 89.0000%(57/64)
Batch[508] - loss: 0.402767  acc: 81.0000%(52/64)
Batch[509] - loss: 0.369449  acc: 8

In [9]:
file = 'nn_train_poetry.csv'
with open(file, 'r', newline='', encoding='utf-8') as myFile:  
    rdr = csv.reader(myFile)
    data = list(rdr)

In [11]:
data[0][0]

'-+-+-+-+-+\t+-++--++-+\t++-+-+-+-+\t-+-+-+++-+\t+-++-+-+-+\t-+-+-+-+-+\t-+-+-++--+\t-+-+-+-+-+\t-+-+-+-+-+\t-+-+-+-+-+\t+-++-+-+-+\t-+-+-+-+-+\t-+-+-+-+-+\t-+++-+-+-+'